In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from io_images import get_images_infoframe
import paramiko
import os 
from process_ims import get_ims_pixel_size

/home/thomas/miniconda3/envs/preprocess-img/lib/python3.10/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "cipher": algorithms.TripleDES,
/home/thomas/miniconda3/envs/preprocess-img/lib/python3.10/site-packages/paramiko/transport.py:258: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "class": algorithms.TripleDES,


Get chunk images .tiff

In [ ]:
windows = "nt" in os.name

In [ ]:
win_path = r"/run/user/1009/gvfs/smb-share:server=fs.ista.ac.at,share=drives/tnegrell/group/siegegrp/FlSu/_Bioimaging/Thomas/chunked/retina_example"
# project_path = "results/retina_example"
infoframe = get_images_infoframe(win_path, 
                                 extension='.tif',
                                 conditions = [],
)

In [ ]:
infoframe

In [ ]:
import re
mu_per_p = 2.7122
mz_per_p = 0.900
# Function to extract boundaries
def extract_boundaries(filename):
    match = re.search(r'x__([\d]+)_([\d]+)__y__([\d]+)_([\d]+)', filename)
    if match:
        return {
            "ExtendMinX": float(match.group(1))*mu_per_p,
            "ExtendMaxX": float(match.group(2))*mu_per_p,
            "ExtendMinY": float(match.group(3))*mu_per_p,
            "ExtendMaxY": float(match.group(4))*mu_per_p,
            "ExtendMinZ": 0.,
            "ExtendMaxZ": 50.*mz_per_p,
            "SizeX": abs(int(match.group(1))-int(match.group(2))),
            "SizeY": abs(int(match.group(3))-int(match.group(4))),
            "SizeZ": 50,


        }
    return {}

# Apply function to create dictionaries
infoframe["boundaries"] = infoframe["file_name"].apply(extract_boundaries)

# Convert to a list of dictionaries if needed
infoframe

In [ ]:
import tifffile as tiff
import json
import numpy as np
import pandas as pd
import os

In [ ]:
import os
import json
import tifffile as tiff

# Ensure output directory exists
os.makedirs(win_path, exist_ok=True)

# Iterate through DataFrame and save TIFF files
for _, row in infoframe.iterrows():
    file_name = "meatdata_" + row["file_name"] 
    metadata = row["boundaries"]
    file_path = row["file_path"]  # Path to original TIFF file

    # Convert metadata dictionary to JSON string
    metadata_json = json.dumps(metadata)

    # Read original TIFF image
    try:
        image_data = tiff.imread(file_path)  # Load image safely
    except FileNotFoundError:
        print(f"Error: File not found - {file_path}")
        continue
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        continue

    # Define output file path (save modified TIFF in win_path)
    output_path = os.path.join(win_path, file_name)

    # Save TIFF file with metadata
    tiff.imwrite(output_path, image_data, description=metadata_json)

    print(f"Saved {output_path} with metadata: {metadata}")


In [ ]:
import tifffile as tiff
    
# Load the TIFF file
array = tiff.imread(infoframe['file_path'].iloc[4])

# Check the shape of the array
print(f"Loaded TIFF shape: {array.shape}")

Get one chunk

In [ ]:
# Define the file name you are searching for
filename = infoframe['file_name'].iloc[4]
filepath = infoframe['file_path'].iloc[4]
# Get the index of the row containing the file name
index = infoframe.loc[infoframe['file_name'] == filename].index

# If you want to print or use the index
print(f"Index of the row: {index}")


get metadata

In [ ]:
# (0.9, 0.339, 0.339)
1/0.339

Get the one

In [ ]:
filename

In [3]:
# base_path = f'/mnt/archive/archive/siegegrp/AlVe/MORPHOMICS2.0_MICROGLIA_BRAIN_ATLAS'
base_path = r"/run/user/1009/gvfs/smb-share:server=fs.ista.ac.at,share=drives/tnegrell/group/siegegrp/FlSu/_Bioimaging/Thomas/chunked/retina_example"
filename = "meatdata_trial_chunk_1_over_8_x__5675_11350__y__0_5675__.tif"

file_path_args = f'-ip "{os.path.join(base_path, filename)}"'

cube_args = f'-wss "(128, 128, 16)" -wsb "(128, 128, 16)" -tp "(2.95, 2.95, 1.1)"'

save_path = 'results/traced_microglia'

save_args = f'-spd "{save_path}" -mp "{save_path}" -spsl "{save_path}" -bsp "{save_path}" -nw 1'

command = f'{function} {file_path_args} {cube_args} {save_args}'


NameError: name 'filename' is not defined

In [ ]:
print(command)

In [ ]:
import subprocess
subprocess.run(command, shell=True, check=True)
